In [ ]:
from google.colab import drive, userdata
import os
import re
import time
from google import genai

# === Mount Google Drive and API ===
drive.mount('/content/drive')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

# === Paths ===
base_dir = "/content/drive/My Drive/Test_28_Adnew_wav/"
asr_dir = os.path.join(base_dir, "testgenmt","Alleng")  # input SRTs
mt_dir = os.path.join(base_dir, "testgenmt","Eng_others", "Ben")  # translated output
srt_dir = os.path.join(mt_dir, "Srt")
txt_dir = os.path.join(mt_dir, "Txt")
os.makedirs(srt_dir, exist_ok=True)
os.makedirs(txt_dir, exist_ok=True)
os.makedirs(mt_dir, exist_ok=True)

target_language = "Bengali"
print("🟢 Ready — Processing all .srt files...")

# === SRT parsing pattern ===
pattern = r"(\d+)\s+([\d:,]+ --> [\d:,]+)\s+(.+?)(?=\n\d+\n|$)"

def translate_batch(batch):
    """Translate batch of subtitle text chunks with Gemini, using segment number."""
    # Prepare the joined text with segment number only (no timestamp)
    joined_text = "\n".join([f"{num} {text}" for num, _, text in batch])

    prompt = f"""
    You are a professional subtitle translator for Indic languages.

    Translate the following subtitle dialogue into {target_language}.
    Preserve meaning. Keep subtitles natural.
    Do NOT translate numbers or timestamps.
    Return one line per subtitle, in order, with the segment number included.

    Text:
    {joined_text}
    """

    # Retry logic in case of error
    for _ in range(3):
        try:
            response = client.models.generate_content(
                model="gemini-2.5-pro",
                contents=prompt
            )
            result_text = response.text.strip()
            # Split the response into lines
            return result_text.split("\n")
        except Exception as e:
            print("Retrying batch due to error:", e)
            time.sleep(3)

    # If all retries fail, return empty lines
    return [""] * len(batch)

# === Loop over all SRT files ===
for f_name in os.listdir(asr_dir):
    if not f_name.lower().endswith(".srt"):
        continue

    input_file = os.path.join(asr_dir, f_name)
    print(f"\n🎬 Processing: {f_name}")

    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    entries = re.findall(pattern, content, flags=re.DOTALL)
    print(f"   → {len(entries)} subtitles detected")

    translated_entries = []
    translated_text_only = []

    batch_size = 10
    for i in range(0, len(entries), batch_size):
        batch = entries[i:i + batch_size]

        # Include segment number alongside the text
        orig_texts = [(t[0], t[1], t[2].strip()) for t in batch]  # Segment number, timestamp, and subtitle text

        # Send the batch to Gemini for translation
        translated_batch = translate_batch(orig_texts)

        # After translation, map back to original format (segment number, timestamp, and translated text)
        for (num, ts, _), trans in zip(orig_texts, translated_batch):
            # Remove the extra segment number from Gemini's response
            trans = trans.replace(f"{num} ", "")  # Remove the segment number from the translation
            translated_entries.append(f"{num}\n{ts}\n{trans}\n")
            translated_text_only.append(trans)

        print(f"   ✅ Translated segments {i+1}–{min(i + batch_size, len(entries))}")

    # Save outputs
    base = os.path.splitext(f_name)[0]
    srt_out = os.path.join(srt_dir, f"{base}_{target_language}.srt")
    txt_out = os.path.join(txt_dir, f"{base}_{target_language}.txt")

    with open(srt_out, "w", encoding='utf-8') as f:
        f.write("\n".join(translated_entries))

    with open(txt_out, "w", encoding='utf-8') as f:
        f.write("\n".join(translated_text_only))

    print(f"   📁 Saved → {srt_out}")
    print(f"   📄 Saved → {txt_out}")

print("\n✅ All files translated successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🟢 Ready — Processing all .srt files...

🎬 Processing: Chapter 10B - Packaging, Labeling & Branding_Gemini25pro_hi_hi.srt_English.srt
   → 98 subtitles detected
